In [ ]:
import json
import os
from pydub import AudioSegment
from tqdm import tqdm
import yaml

In [ ]:
SILENCE_RATE = 2

CATEGORY = yaml.safe_load(open("STEP_0_define_category_sub_category.yaml", "r"))['category']
SUB_CATEGORY = yaml.safe_load(open("STEP_0_define_category_sub_category.yaml", "r"))['sub_category']
print(f"Category: {CATEGORY}")
print(f"Sub Category: {SUB_CATEGORY}")

In [ ]:
data = json.load(open(f'category_data/step_1/{CATEGORY}_{SUB_CATEGORY}.json'))

In [ ]:
def get_duration(path):
    try:
        audio = AudioSegment.from_file(path)  # Replace with your file path
        duration_seconds = audio.duration_seconds
        return duration_seconds
    except Exception as e:
        print(f"Error reading audio file: {e}")
        return None

In [ ]:
for sample_i, sample_obj in tqdm(enumerate(data), total=len(data)):
    for lang in 'english', 'italian':
        path = sample_obj[lang]['audio_file']
        assert(os.path.exists(path))
        duration = get_duration(path) 
        data[sample_i][lang]['duration_ms'] = int(duration * 1000)

In [ ]:
def combine_audio_files(input_paths, output_path):
    combined_audio = None
    for path in input_paths:
        if combined_audio is None:
            combined_audio = AudioSegment.from_file(path)
        else:
            combined_audio += AudioSegment.from_file(path)
    combined_audio.export(output_path, format="mp3")

In [ ]:
for sample_i, sample_obj in tqdm(enumerate(data), total=len(data)):
    english_path = sample_obj['english']['audio_file']
    italian_path = sample_obj['italian']['audio_file']
    italian_duration = sample_obj['italian']['duration_ms']
    silence_duration = int(((italian_duration * SILENCE_RATE) // 100) * 100)
    silence_path = f'data/audio/silence/{silence_duration}ms.mp3'
    assert os.path.exists(silence_path), f"Silence file {silence_path} does not exist."
    combined_audio_path = f'category_data/audio/{CATEGORY}_{SUB_CATEGORY}/{sample_i}.mp3'
    os.makedirs(os.path.dirname(combined_audio_path), exist_ok=True)
    audio_files = [english_path, silence_path, italian_path, silence_path, italian_path, silence_path]
    if not os.path.exists(combined_audio_path):
    # if True:
        combine_audio_files(audio_files, combined_audio_path)
    else:
        print(f"Skipping {combined_audio_path} as it already exists.")
    data[sample_i]['combined_audio_file'] = combined_audio_path

In [ ]:
json.dump(data, open(f"category_data/step_2/{CATEGORY}_{SUB_CATEGORY}.json", 'w'), indent=2, ensure_ascii=False)